In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('original/Cleaned_Data-10-24-23.xlsx')
print(df.shape)
df.head(3)

(7767, 26)


,SMILES,Name,Extractant Concentration (M),Solvent,Metal Identity,Oxidation Number,Isotope,Acid Type,Acid Concentration (M),Add. Ligand,...,Formula,Molecular Mass (g/mol),Chemical Group,Additional,InChI,InChIKey,Removed Values from Du,Date,Reference,Link
0,S=P(S)(c1ccccc1Cl)c1ccccc1Cl,(ClPh)2PSSH,0.5,toluene,Am,III,NaN,HNO3,0.2,TBP,...,C12H9Cl2PS2,319.198,NaN,NaN,"InChI=1S/C12H9Cl2PS2/c13-9-5-1-3-7-11(9)15(16,...",KZFPJCGEOBHFRG-UHFFFAOYSA-N,&ap;,2023-10-24,doi:10.1524/ract.2010.1708,https://www.oecd-nea.org/ideal/extractants/(Cl...
1,S=P(S)(c1ccccc1Cl)c1ccccc1Cl,(ClPh)2PSSH,0.5,toluene,Am,III,NaN,HNO3,0.2,TBP,...,C12H9Cl2PS2,319.198,NaN,NaN,"InChI=1S/C12H9Cl2PS2/c13-9-5-1-3-7-11(9)15(16,...",KZFPJCGEOBHFRG-UHFFFAOYSA-N,&ap;,2023-10-24,doi:10.1524/ract.2010.1708,https://www.oecd-nea.org/ideal/extractants/(Cl...
2,S=P(S)(c1ccccc1Cl)c1ccccc1Cl,(ClPh)2PSSH,0.5,toluene,Am,III,NaN,HNO3,0.2,TBP,...,C12H9Cl2PS2,319.198,NaN,NaN,"InChI=1S/C12H9Cl2PS2/c13-9-5-1-3-7-11(9)15(16,...",KZFPJCGEOBHFRG-UHFFFAOYSA-N,&ap;,2023-10-24,doi:10.1524/ract.2010.1708,https://www.oecd-nea.org/ideal/extractants/(Cl...


### Filter metals

In [3]:
Ln_list = ['La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu']
metal_pattern = '|'.join(Ln_list)
df_filtered = df[df['Metal Identity'].str.contains(metal_pattern, na=False, regex=True)]

df_filtered.shape

(3123, 26)

### Remove Nan or empty

In [4]:
# Remove rows where there is 'Add. Ligand'
df_filtered = df_filtered[df_filtered['Add. Ligand'].isna()]

print(df_filtered.shape)

(2483, 26)


In [5]:
# Remove rows where 'Distribution Coefficient' is NaN or zero
df_filtered['Distribution Coefficient'] = pd.to_numeric(df_filtered['Distribution Coefficient'], errors='coerce')
df_filtered = df_filtered[df_filtered['Distribution Coefficient'].notna() & (df_filtered['Distribution Coefficient'] > 0)]

print(df_filtered.shape)

(2474, 26)


In [6]:
# Remove rows where 'Temperature (°C)' is NaN

df_filtered['Temperature (°C)'] = pd.to_numeric(df_filtered['Temperature (°C)'], errors='coerce')
df_filtered = df_filtered[df_filtered['Temperature (°C)'].notna()]

print(df_filtered.shape)

(2415, 26)


In [7]:
# Remove rows where 'Extractant Concentration (M)' is NaN
df_filtered['Extractant Concentration (M)'] = pd.to_numeric(df_filtered['Extractant Concentration (M)'], errors='coerce')
df_filtered = df_filtered[df_filtered['Extractant Concentration (M)'].notna()]

df_filtered.shape

(2027, 26)

In [8]:
# Remove rows where 'Acid Concentration (M)' is NaN
df_filtered['Acid Concentration (M)'] = pd.to_numeric(df_filtered['Acid Concentration (M)'], errors='coerce')
df_filtered = df_filtered[df_filtered['Acid Concentration (M)'].notna()]

df_filtered.shape

(1868, 26)

In [9]:
# Keep rows where III or empty
df_filtered = df_filtered[
    (df_filtered['Oxidation Number'] == 'III') | (df_filtered['Oxidation Number'].isna())
]

df_filtered.shape

(1838, 26)

### Calculation

In [10]:
from rdkit import Chem

In [11]:
def get_canonical_smiles(smiles):
    molecule = Chem.MolFromSmiles(smiles)
    return Chem.MolToSmiles(molecule, canonical=True)

df_filtered = df_filtered.rename(columns={'SMILES': 'SMILES_orig'})
df_filtered['SMILES'] = df_filtered['SMILES_orig'].apply(get_canonical_smiles)

df_filtered.shape

(1838, 27)

In [12]:
# Convert T to Kelvin and create a new column 'Temperature_K'
df_filtered['Temperature_K'] = df_filtered['Temperature (°C)'] + 273.15

df_filtered.shape

(1838, 28)

In [13]:
# Calculate the base-10 logarithm and create a new column 'log_D'
df_filtered['log_D'] = np.log10(df_filtered['Distribution Coefficient'])

df_filtered.shape

(1838, 29)

### Parse solvent

In [14]:
import re

In [15]:
# remove rows where the solvent mixture doesn't have ratio

# Define the regex pattern for volume ratio 'a:b'
ratio_pattern = r'\d+(\.\d+)?\s*:\s*\d+(\.\d+)?'

# Filter rows
df_filtered = df_filtered[
    df_filtered['Solvent'].notna() & (
        # Keep rows where 'solvent' does not contain '/' (single solvent)
        (~df_filtered['Solvent'].str.contains('/', na=False)) |
        # Or where 'solvent' contains a volume ratio 'a:b'
        df_filtered['Solvent'].str.contains(ratio_pattern, na=False)
    )
]

print(df_filtered.shape)

(1492, 29)


C:\Users\26070\AppData\Local\Temp\ipykernel_14812\2873118876.py:12: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_filtered['Solvent'].str.contains(ratio_pattern, na=False)


In [16]:
# Parse the 'Solvent' column

def parse_solvent(solvent_str):
    if pd.isna(solvent_str):
        return pd.Series({'Solvent A': None, 'Solvent B': None, 'volume ratio': 0})
    solvent_str = solvent_str.strip()
    # Check if there's a '/' indicating mixed solvents
    if '/' in solvent_str:
        # Split the solvents and the volume ratio
        if ',' in solvent_str:
            # Split at the comma to separate solvents and volume ratio
            solvents_part, ratio_part = solvent_str.split(',', 1)
            
            solvents_part = solvents_part.strip()
            ratio_part = ratio_part.strip()
            # Split solvents
            solvents = solvents_part.split('/', 1)
            if len(solvents) == 2:
                solvent_a = solvents[0].strip()
                solvent_b = solvents[1].strip()
            else:
                solvent_a = solvents_part.strip()
                solvent_b = None
            volume_ratio = float(ratio_part.split(':')[1])/10
    else:
        # Pure solvent
        solvent_a = solvent_str
        solvent_b = None
        volume_ratio = 0
    return pd.Series({'Solvent A': solvent_a, 'Solvent B': solvent_b, 'volume ratio': volume_ratio})

# Apply the function to the 'Solvent' column and create a new DataFrame
df_solvents = df_filtered['Solvent'].apply(parse_solvent)

# Concatenate the new columns with df_filter
df_cleaned = pd.concat([df_filtered, df_solvents], axis=1)

df_cleaned['volume fraction B'] = pd.to_numeric(df_cleaned['volume ratio'], errors='coerce')

df_cleaned.shape

(1492, 33)

In [17]:
# Replace 'doi:' with 'https://doi.org/'

# Function to transform DOI entries
def format_doi(doi):
    if pd.isnull(doi):
        return None
    doi = doi.strip()
    if doi.startswith('doi:'):
        # Replace 'doi:' with 'https://doi.org/'
        return 'https://doi.org/' + doi[len('doi:'):]
    elif doi.startswith('http'):
        return doi
    else:
        # Directly add 'https://doi.org/' if 'doi:' is not present
        return 'https://doi.org/' + doi

df_cleaned['DOI'] = df_cleaned['Reference'].apply(format_doi)

df_cleaned.shape

(1492, 34)

In [18]:
df_cleaned.to_excel('output_cleaned_data.xlsx', index=False)